# Building a ME-model

In [1]:
from __future__ import print_function

import cobrame
from cobrame.util import dogma, building
import cobrame.util.building
import cobra
import cobra.test
from collections import defaultdict

#import warnings
#warnings.filterwarnings('ignore')

/home/sbrg-cjlloyd/cobrapy/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/sbrg-cjlloyd/cobrapy/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


## Overview

COBRAme is constructed entirely over COBRApy. This means that ME-model reactions will have all of the same properties, methods, and functions as a COBRApy reaction. However, one key difference between M and ME models is that many reactions involved in gene expression are effecively templates that are constructed identically but vary based on characteristics of the gene being expressed. For example, a gene with a given nucleotide sequence is always translated following the same rules provided by the codon table for that organism.

In order to facilliate the template nature of many gene expression reactions, COBRAme reactions are constructed and their components are manipulated through the use of `ProcessData` classes. These act as information vessels for holding the information assocatied with a cellular process in simple, standard datatypes such as dictionaries and strings. 

-----

This tutorial will go step-by-step through the process of creating a generic enzyme catalyzed reaction (i.e. metabolic reaction):

which requires the formation and coupling of **complex\_ab** in order to proceed.

-------

In order for this reaction to carry flux in the model we will additionally need to first add the corresponding:
 
 1. **Transcription reactions**
 2. **Translation reactions**
 3. **tRNA charging reactions**
 4. **Complex formation reactions**

Once these are added we will add in the synthesis of key macromolecular components (ribosome, RNA polymerase, etc.) and show how they are coupled to their respective reactions. The derived coupling coefficients will also be described. For more on the derivation of the coupling coefficients, reference the supplemental text of [O'brien et. al. 2013](https://www.ncbi.nlm.nih.gov/pubmed/24084808)

## Initializing new ME-Models

When applying some constraints in the ME-model, metabolite properties are required. For instance, to calculate the total biomass (by molecular weight) produced by a particular macromolecule, the amino acid, nucleotide, etc. molecular weights are required. To enable these calculations, all metabolites from *i*JO1366, along with their metabolite attributes are added to the newly initialized ME-model.

Further the reactions from *i*JO1366 will be added to the ME-model to demonstrate ME-model solving procedures.

In [2]:
# create empty ME-model
me = cobrame.MEModel('test')
ijo = cobra.test.create_test_model('ecoli')

In [3]:
# Add all metabolites and reactions from iJO1366 to the new ME-model
for met in ijo.metabolites:
    me.add_metabolites(met)
for rxn in ijo.reactions:
    me.add_reaction(rxn)

The ME-model contains a "global_info" attribute which stores information used to calculate coupling constraints, along with other functions. The specifics of each of these constraints will be discussed when they are implemented.
<br>
<br>

<div class="alert alert-info">
**Note**: k_deg will initially be set to 0. We will apply RNA degradation later in the tutorial.
</div>

In [4]:
# "Translational capacity" of organism
me.global_info['kt'] = 4.5  # (in h-1)scott 2010, RNA-to-protein curve fit
me.global_info['r0'] = 0.087  # scott 2010, RNA-to-protein curve fit
me.global_info['k_deg'] = 1.0/5. * 60.0  # 1/5 1/min 60 min/h # h-1

# Molecular mass of RNA component of ribosome
me.global_info['m_rr'] = 1453. # in kDa

# Average molecular mass of an amino acid
me.global_info['m_aa'] = 109. / 1000.  # in kDa

# Proportion of RNA that is rRNA
me.global_info['f_rRNA'] = .86
me.global_info['m_nt'] = 324. / 1000.  # in kDa
me.global_info['f_mRNA'] = .02

# tRNA associated global information
me.global_info['m_tRNA'] = 25000. / 1000.  # in kDA
me.global_info['f_tRNA'] = .12

# Define the types of biomass that will be synthesized in the model
me.add_biomass_constraints_to_model(["protein_biomass", "mRNA_biomass", "tRNA_biomass", "rRNA_biomass",
                                     "ncRNA_biomass", "DNA_biomass", "lipid_biomass", "constituent_biomass",
                                     "prosthetic_group_biomass", "peptidoglycan_biomass"])

Define sequence of gene that will be expressed in tutorial

In [5]:
sequence = ("ATG" + "TTT" * 12 + "TAT" * 12 + 
            "ACG" * 12 + "GAT" * 12 + "AGT" * 12 + "TGA")

## Adding Reactions without utility functions
We'll first demonstrate how transcription, translation, tRNA charging, complex formation, and metabolic reactions can be added to a model without using any of the utility functions provided in `cobrame.util.building.py`. The second half of the tutorial will show how these utility functions can be used to add these reactions.

The basic workflow for adding any reaction to a ME-model using COBRAme occurs in three steps:

1. **Create the ProcessData(s) associated with the reaction and populate them with the necessary information**
2. **Create the MEReaction and link the appropriate ProcessData**
3. **Execute the MEReaction's update method**

### Add Transcription Reaction

#### Add TranscribedGene metabolite to model
Transcription reactions is unique in that they occur at a transcription unit level and can code for multiple transcript products. Therefore the nucleotide sequence of both the transcription unit and the RNA transcripts must be defined in order to correctly construct a transcription reaction.

In [6]:
gene = cobrame.TranscribedGene('RNA_a', 'mRNA', sequence)
me.add_metabolites([gene])

When adding the `TranscribedGene` above, the `RNA_type` and `nucleotide_sequence` was assigned to the gene. This sequence cannot be determined from the transcription unit (TU) sequence because a single TU often contains several different RNAs.

#### Add TranscriptionData to model

In [7]:
transcription_data = cobrame.TranscriptionData('TU_a',me,rna_products={'RNA_a'})
transcription_data.nucleotide_sequence = sequence

#### Add TranscriptionReaction to model
And point `TranscriptionReaction` to `TranscriptionData`

In [8]:
transcription_rxn = cobrame.TranscriptionReaction('transcription_TU_a')
transcription_rxn.transcription_data = transcription_data
me.add_reactions([transcription_rxn])

#### Update TranscriptionReaction

In [9]:
transcription_rxn.update()
print(transcription_rxn.reaction)

86 atp_c + 38 ctp_c + 12 gtp_c + 50 utp_c --> RNA_a + 59.172286 mRNA_biomass + 186 ppi_c


/home/sbrg-cjlloyd/cobrame/cobrame/core/reaction.py:813 UserWarning: RNA Polymerase () not found


<div class="alert alert-info">
**Note**: the RNA_polymerase complex is not included in the reaction. This will be added later
</div>

This reaction now produces a small amount of the a `mRNA_biomass` metabolite (constraint). This term has a coefficient corresponding to the molecular weight (in $kDA$) of the RNA being transcribed. This constraint will be implemented into a $v_{biomasss\_dilution}$ reaction with the form: 
$$\mu \leq v_{biomass\_dilution} \leq \mu $$

A mathematical description of the `biomass` constraint can be found in *Biomass Dilution Constraints* in **ME-Model Fundamentals**.

<div class="alert alert-info">
**Note**: This is not a complete picture of transcription because the RNA polymerase is missing.
</div>

#### Incorporate RNA Polymerase
For the purposes of this tutorial, we'll skip the steps required to add the reactions to form the RNA_polymerase. The steps however are identical to those outlined in **add enzyme complexes** below

In [10]:
RNAP = cobrame.RNAP('RNA_polymerase')
me.add_metabolites(RNAP)

Associate RNA_polymerase with all `TranscriptionData` and update

In [11]:
for data in me.transcription_data:
    data.RNA_polymerase = RNAP.id
me.reactions.transcription_TU_a.update()

print(me.reactions.transcription_TU_a.reaction)

0.00088887053605567*mu + 0.000347992814865795 RNA_polymerase + 86 atp_c + 38 ctp_c + 12 gtp_c + 50 utp_c --> RNA_a + 59.172286 mRNA_biomass + 186 ppi_c


The coefficient for RNA_polymerase is the first instance in this tutorial where a coupling constraint is imposed. In this case the constraint couples the formation of a RNA_polymerase metabolite to its transcription flux. This constraint is formulated as in [O'brien et. al. 2013](https://www.ncbi.nlm.nih.gov/pubmed/24084808), with assumption that $k_{rnap} = 3 \cdot k_{ribosome}$ based on data from [Proshkin et al. 2010](https://www.ncbi.nlm.nih.gov/pubmed/20413502):

$$
\begin{align}
v_{dilution,RNAP, j} =  \frac{l_{TU,j}}{3 c_{ribo}\kappa_{\tau}} v_{transcription,j} (\mu+r_0\kappa_{\tau}), & \forall j \in TU
\end{align}
$$

where:

 - $\kappa_{\tau}$ and $r_0$ are phenomenological parameters from [Scott et. al. 2010](https://www.ncbi.nlm.nih.gov/pubmed/21097934) that describe the linear relationship between the observed RNA/protein ratio of *E. coli* and its growth rate ($\mu$)

 - $c_{ribo} = \frac{m_{rr}}{f_{rRNA}\cdot m_{aa}}$ where: $m_{rr}$ is the mass of rRNA per ribosome. $f_{rRNA}$ is the fraction of total RNA that is rRNA $m_{aa}$ is the molecular weight of an average amino acid

 - $v_{transcription, j}$ is the rate of transcription for $TU_j$
 
 - $l_{TU, j}$ is number of nucleotides in $TU_j$
 
----
### Add Translation Reaction
#### Add TranslationData to model
In order to add a TranslationData object to a ME-model the user must additionally specifify the mRNA id and protein id of the translation reaction that will be added. This information as well as a nucleotide sequence is the only information required to add a translation reaction.

In [12]:
data = cobrame.TranslationData('a', me, 'RNA_a', 'protein_a')
data.nucleotide_sequence = sequence

#### Add TranslationReaction to model
By associating the TranslationReaction with its corresponding TranslationData object and running the update function, COBRAme will create a reaction reaction for the nucleotide sequence given based on the organisms codon table and prespecified translation machinery.

In [13]:
rxn = cobrame.TranslationReaction('translation_a')
rxn.translation_data = data
me.add_reaction(rxn)

#### Update TranslationReaction

In [14]:
rxn.update()
print(rxn.reaction)

/home/sbrg-cjlloyd/cobrame/cobrame/core/reaction.py:1051 UserWarning: ribosome not found
/home/sbrg-cjlloyd/cobrame/cobrame/core/reaction.py:1094 UserWarning: RNA_degradosome not found


0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_a + 12 asp__L_c + 0.276611796982167*(mu + 0.3915)/mu atp_c + 0.353897348367627*(mu + 0.3915)/mu mRNA_biomass + met__L_c + 12 phe__L_c + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> 0.276611796982167*(mu + 0.3915)/mu adp_c + 0.514348422496571*(mu + 0.3915)/mu amp_c + 0.227270233196159*(mu + 0.3915)/mu cmp_c + 0.0717695473251029*(mu + 0.3915)/mu gmp_c + 60.0 - 0.276611796982167*(mu + 0.3915)/mu h2o_c + 0.276611796982167*(mu + 0.3915)/mu h_c + 0.276611796982167*(mu + 0.3915)/mu pi_c + protein_a + 7.500606 protein_biomass + 0.299039780521262*(mu + 0.3915)/mu ump_c


In this case the constraint couples the formation of a mRNA metabolite to its translation flux. This constraint is formulated as in [O'brien et. al. 2013](https://www.ncbi.nlm.nih.gov/pubmed/24084808):

$$
\begin{align}
v_{dilution,j} = \frac{3}{\kappa_{\tau} c_{mRNA}} \cdot (\mu + \kappa_{\tau} r_0) v_{translation,j}  , & &  \forall j \in mRNA
\end{align}
$$

where:

 - $\kappa_{\tau}$ and $r_0$ are phenomenological parameters from [Scott et. al. 2010](https://www.ncbi.nlm.nih.gov/pubmed/21097934) that describe the linear relationship between the observed RNA/protein ratio of *E. coli* and its growth rate ($\mu$)

 - $c_{mRNA} = \frac{m_{nt}}{f_{mRNA}\cdot m_{aa}}$ where: $m_{nt}$ is the molecular weight of an average mRNA nucleotide. $f_{mRNA}$ is the fraction of total RNA that is mRNA $m_{aa}$ is the molecular weight of an average amino acid

 - $v_{translation, j}$ is the rate of translation for $mRNA_j$

#### Incorporate Ribosome

In [15]:
ribosome = cobrame.Ribosome('ribosome')
me.add_metabolites([ribosome])
me.reactions.translation_a.update()
print(me.reactions.translation_a.reaction)

0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_a + 12 asp__L_c + 0.276611796982167*(mu + 0.3915)/mu atp_c + 0.353897348367627*(mu + 0.3915)/mu mRNA_biomass + met__L_c + 12 phe__L_c + 0.000874533914506385*mu + 0.00034238002752925 ribosome + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> 0.276611796982167*(mu + 0.3915)/mu adp_c + 0.514348422496571*(mu + 0.3915)/mu amp_c + 0.227270233196159*(mu + 0.3915)/mu cmp_c + 0.0717695473251029*(mu + 0.3915)/mu gmp_c + 60.0 - 0.276611796982167*(mu + 0.3915)/mu h2o_c + 0.276611796982167*(mu + 0.3915)/mu h_c + 0.276611796982167*(mu + 0.3915)/mu pi_c + protein_a + 7.500606 protein_biomass + 0.299039780521262*(mu + 0.3915)/mu ump_c


/home/sbrg-cjlloyd/cobrame/cobrame/core/reaction.py:1094 UserWarning: RNA_degradosome not found


This imposes a new coupling constraint for the ribosome. In this case the constraint couples the formation of a ribosome to its translation flux. This constraint is formulated as in [O'brien et. al. 2013](https://www.ncbi.nlm.nih.gov/pubmed/24084808):

$$
\begin{align}
v_{dilution,ribo, j} = \frac{l_{p,j}}{c_{ribo}\kappa_{\tau}} v_{translation,j} (\mu+r_0\kappa_{\tau}) , & \forall j \in mRNA
\end{align}
$$

where:

 - $\kappa_{\tau}$ and $r_0$ are phenomenological parameters from [Scott et. al. 2010](https://www.ncbi.nlm.nih.gov/pubmed/21097934) that describe the linear relationship between the observed RNA/protein ratio of *E. coli* and its growth rate ($\mu$)

 - $c_{ribo} = \frac{m_{rr}}{f_{rRNA}\cdot m_{aa}}$ where: $m_{nt}$ is the mass of rRNA per ribosome. $f_{rRNA}$ is the fraction of total RNA that is rRNA $m_{aa}$ is the molecular weight of an average amino acid

 - $v_{translation, j}$ is the rate of translation for $mRNA_j$
 
 - $l_{p, j}$ is number of amino acids in peptide translated from $mRNA_j$
 
-------
<div class="alert alert-info">
**Note**: The above reactions do not provide a complete picture of translation in that it is missing charged tRNAs to facillitate tRNA addition.
</div>

Below, we'll correct this by adding in an tRNA charging reaction.

### Add tRNA Charging Reaction

#### Add tRNAData to model

In [16]:
# Must add tRNA metabolite first
gene = cobrame.TranscribedGene('RNA_d', 'tRNA', sequence)
me.add_metabolites([gene])

In [17]:
data = cobrame.tRNAData('tRNA_d_GUA', me, 'val__L_c', 'RNA_d', 'GUA')

#### Add tRNAChargingReaction to model
And point `tRNAChargingReaction` to `tRNAData`

In [18]:
rxn = cobrame.tRNAChargingReaction('charging_tRNA_d_GUA')
me.add_reaction(rxn)
rxn.tRNA_data = data

#### Update tRNAChargingReaction

In [19]:
#Setting verbose=False suppresses print statements indicating that new metabolites were created
rxn.update(verbose=False)
print(rxn.reaction)

0.000116266666666667*mu + 4.55184e-5 RNA_d + 0.000116266666666667*mu + 4.55184e-5 val__L_c --> generic_tRNA_GUA_val__L_c


This reaction creates one ```generic_charged_tRNA``` equivalement that can then be used in a translation reaction

The coefficient for ```RNA_d``` and ```lys__L_c``` are defined by:

$$
\begin{align}
v_{dilution,j} \geq \frac{1}{\kappa_{\tau} c_{tRNA,j}} (\mu + \kappa_{\tau} r_0)  v_{charging,j} , & \forall j \in tRNA
\end{align}
$$

where:

 - $\kappa_{\tau}$ and $r_0$ are phenomenological parameters from [Scott et. al. 2010](https://www.ncbi.nlm.nih.gov/pubmed/21097934) that describe the linear relationship between the observed RNA/protein ratio of *E. coli* and its growth rate ($\mu$)

 - $c_{tRNA, j} = \frac{m_{tRNA}}{f_{tRNA}\cdot m_{aa}}$ where: $m_{tRNA}$ is molecular weight of an average tRNA. $f_{tRNA}$ is the fraction of total RNA that is tRNA $m_{aa}$ is the molecular weight of an average amino acid

 - $v_{charging, j}$ is the rate of charging for $tRNA_j$
 
<div class="alert alert-info">
**Note**: This tRNA charging reaction is still missing a tRNA synthetase which catalyzes the amino acid addition to the uncharged tRNA.
</div>

#### Incorporate tRNA Synthetases

In [20]:
synthetase = cobrame.Complex('synthetase')
me.add_metabolites(synthetase)

Associate synthetase with `tRNAData` and update

In [21]:
data.synthetase = synthetase.id
rxn.update()
print(rxn.reaction)

0.000116266666666667*mu + 4.55184e-5 RNA_d + 4.27350427350427e-6*mu*(0.000116266666666667*mu + 1.0000455184) synthetase + 0.000116266666666667*mu + 4.55184e-5 val__L_c --> generic_tRNA_GUA_val__L_c


The synthetase coupling was reformulated from [O'brien et. al. 2013](https://www.ncbi.nlm.nih.gov/pubmed/24084808) enable more modularity in the ME-model. A more complete mathematical description of the tRNA synthetase coupling constraints can be found in the [tRNA.ipynb](tRNA.ipynb)

---

### Add tRNAs to Translation

Here we take advantage of an additional subclass of `ProcessData`, called a `SubreactionData` object. This class is used to lump together processeses that occur as a result of many individual reactions, including translation elongation, ribosome formation, tRNA modification, etc. Since each of these steps often involve an enzyme that requires its own coupling constraint, this process allows these processes to be lumped into one reaction while still enabling each subprocess to be modified.

`TranslationData` objects have an `subreaction_from_sequence` method that returns any subreactions that have been added to the model and are part of translation elongation (i.e. tRNA). Since no tRNA-mediated amino acid addition subreactions have been added to the model, the below call returns nothing.

In [22]:
print(me.process_data.a.subreactions_from_sequence)

{}


/home/sbrg-cjlloyd/cobrame/cobrame/core/processdata.py:826 UserWarning: tRNA addition subreaction phe_addition_at_UUU not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/processdata.py:826 UserWarning: tRNA addition subreaction tyr_addition_at_UAU not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/processdata.py:826 UserWarning: tRNA addition subreaction thr_addition_at_ACG not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/processdata.py:826 UserWarning: tRNA addition subreaction asp_addition_at_GAU not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/processdata.py:826 UserWarning: tRNA addition subreaction ser_addition_at_AGU not in model


`UserWarnings` are returned to indicate that tRNA subreactions have not been added for each codon.

Below, we add the SubreactionData (excluding enzymes) for the addition of an amino acid using information from the *E. coli* codon table. The charge tRNA does not act as an enzyme in this case because it's coupling is handled in the `tRNAChargingReaction`

#### Add Subreactions for tRNA addition to model

In [23]:
data = cobrame.SubreactionData('asp_addition_at_GAU', me)
data.stoichiometry = {'generic_tRNA_GAU_asp__L_c': -1,
                      'gtp_c': -1, 'gdp_c': 1, 'h_c': 1,
                      'pi_c': 1}

Now calling `subreactions_from_sequence` returns the number of tRNA subreactions that should be added to the `TranslationData`

In [24]:
translation_subreactions = me.process_data.a.subreactions_from_sequence
print(translation_subreactions)

{'asp_addition_at_GAU': 12}


/home/sbrg-cjlloyd/cobrame/cobrame/core/processdata.py:826 UserWarning: tRNA addition subreaction phe_addition_at_UUU not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/processdata.py:826 UserWarning: tRNA addition subreaction tyr_addition_at_UAU not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/processdata.py:826 UserWarning: tRNA addition subreaction thr_addition_at_ACG not in model
/home/sbrg-cjlloyd/cobrame/cobrame/core/processdata.py:826 UserWarning: tRNA addition subreaction ser_addition_at_AGU not in model


Updating `TranslationData.subreactions` with the tRNA subreactions incorporates this information into the `TranslationReaction`

In [25]:
print("Before adding tRNA subreaction")
print("------------------------------")
print(me.reactions.translation_a.reaction)
print("")
# Link tranlation_data to subreactions and update
for subreaction, value in translation_subreactions.items():
    me.process_data.a.subreactions[subreaction] = value
me.reactions.translation_a.update(verbose=False)
print("After adding tRNA subreaction")
print("-----------------------------")
print(me.reactions.translation_a.reaction)

Before adding tRNA subreaction
------------------------------
0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_a + 12 asp__L_c + 0.276611796982167*(mu + 0.3915)/mu atp_c + 0.353897348367627*(mu + 0.3915)/mu mRNA_biomass + met__L_c + 12 phe__L_c + 0.000874533914506385*mu + 0.00034238002752925 ribosome + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> 0.276611796982167*(mu + 0.3915)/mu adp_c + 0.514348422496571*(mu + 0.3915)/mu amp_c + 0.227270233196159*(mu + 0.3915)/mu cmp_c + 0.0717695473251029*(mu + 0.3915)/mu gmp_c + 60.0 - 0.276611796982167*(mu + 0.3915)/mu h2o_c + 0.276611796982167*(mu + 0.3915)/mu h_c + 0.276611796982167*(mu + 0.3915)/mu pi_c + protein_a + 7.500606 protein_biomass + 0.299039780521262*(mu + 0.3915)/mu ump_c

After adding tRNA subreaction
-----------------------------
0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_a + 12 asp__L_c + 0.276611796982167*(mu + 0.3915)/mu atp_c + 12.0 generic_

/home/sbrg-cjlloyd/cobrame/cobrame/core/reaction.py:1094 UserWarning: RNA_degradosome not found
/home/sbrg-cjlloyd/cobrame/cobrame/core/processdata.py:229 UserWarning: No element contribution input for subreaction (asp_addition_at_GAU), calculating based on stoichiometry instead


### Add Complex Formation Reaction
#### Add ComplexData to model
For COBRAme models, the reaction gene-protein-reaction rule (GPR) is replaced with a metabolite representing the synthesis of the enzyme(s) catalyzing a reaction. This metabolite is formed explicitly in a ME model by seperate reaction to transcribe the gene(s) and translate the protein(s) the compose the complex. 

In [26]:
data = cobrame.ComplexData('complex_ab', me)
data.stoichiometry = {'protein_a': 1, 'protein_b': 1}

#### Add  ComplexFormation reaction to model
And point `ComplexFormation` to `ComplexData`

In [27]:
rxn = cobrame.ComplexFormation('formation_complex_ab')
me.add_reaction(rxn)
rxn.complex_data_id = data.id
rxn._complex_id = data.id

#### Update ComplexFormation reaction

In [28]:
rxn.update(verbose=False)
print(me.reactions.formation_complex_ab.reaction)

protein_a + protein_b --> complex_ab


#### Apply modification to complex formation reaction
Many enzyme complexes in an ME-model require cofactors or prosthetic groups in order to properly function. Information about such processes are stored as ModificationData.

For instance, we can add the modification of an iron-sulfur cluster, a common prosthetic group, by doing the following:

In [29]:
# Define the stoichiometry of the modification 
mod_data = cobrame.SubreactionData('mod_2fe2s_c', me)
mod_data.stoichiometry = {'2fe2s_c': -1}
# this process can also be catalyzed by a chaperone
mod_data.enzyme = 'complex_ba'
mod_data.keff = 65.  # default value

Associate modification to complex and ```update()``` its formation

In [30]:
complex_data = me.process_data.complex_ab
complex_data.subreactions['mod_2fe2s_c'] = 1

Update ComplexFormation reaction

In [31]:
print('Before adding modification')
print('--------------------------')
print(me.reactions.formation_complex_ab.reaction)
me.reactions.formation_complex_ab.update()
print('\n')
print('After adding modification')
print('-------------------------')
print(me.reactions.formation_complex_ab.reaction)

Before adding modification
--------------------------
protein_a + protein_b --> complex_ab
Created <Complex complex_ba at 0x7f4bf69a8b38> in <ComplexFormation formation_complex_ab at 0x7f4bf5f13ef0>


After adding modification
-------------------------
2fe2s_c + 4.27350427350427e-6*mu complex_ba + protein_a + protein_b --> complex_ab + 0.17582 prosthetic_group_biomass


### Add Metabolic Reaction
#### Add StoichiometricData to model
MetabolicReactions require, at a minimum, one corresponding StoichiometricData. StoichiometricData essentially holds the information contained in an M-model reaction. This includes the metabolite stoichiometry and the upper and lower bound of the reaction. As a best practice, StoichiometricData typically uses an ID equivalent to the M-model reaction ID.

So first, we will create a StoichiometricData object to define the stoichiometry of the conversion of *a* to *b*. **Only one StoichiometricData object should be created for both reversible and irreversible reactions**

In [32]:
# unique to COBRAme, construct a stoichiometric data object with the reaction information
data = cobrame.StoichiometricData('a_to_b', me)
stoichiometry = {'a':-1, 'b': 1}
data._stoichiometry = stoichiometry
data.lower_bound = -1000
data.upper_bound = 1000

#### Add MetabolicReaction to model

The StoichiometricData for this reversible reaction is then assigned to two different MetabolicReactions (Due to the enzyme dilution constraint, all enzyme catalyzed reactions must be reverisble; more on this later). The MetabolicReactions require:
 - The associated StoichiometricData
 - The *reverse* flag set to True for reverse reactions, False for forward reactions
 - Enzyme $K_{eff}$ for reaction (discussed later, dafault=65)

These fields are then processed and the actual model reaction is created using the MetabolicReaction's update() function

In [33]:
# Create a forward ME Metabolic Reaction and associate the stoichiometric data to it
rxn_fwd = cobrame.MetabolicReaction('a_to_b_FWD_complex_ab')
me.add_reaction(rxn_fwd)
rxn_fwd.stoichiometric_data = data
rxn_fwd.reverse = False
rxn_fwd.keff = 65.

# Create a reverse ME Metabolic Reaction and associate the stoichiometric data to it
rxn_rev = cobrame.MetabolicReaction('a_to_b_REV_complex_ab')
me.add_reaction(rxn_rev)
rxn_rev.stoichiometric_data = data
rxn_rev.reverse = True
rxn_rev.keff = 65.

#### Update MetabolicReactions

In [34]:
rxn_fwd.update(verbose=False)
rxn_rev.update(verbose=False)
print(me.reactions.a_to_b_FWD_complex_ab.reaction)
print(me.reactions.a_to_b_REV_complex_ab.reaction)

a --> b
b --> a


<div class="alert alert-info">
**Note**: the $k_{eff}$ and `complex_ab` is not included in the reaction since no complex has been associated to it yet
</div>

#### Associate enzyme with MetabolicReaction

The ComplexData object created in the previous cell can be incorporated into the MetabolicReaction using code below. 

<div class="alert alert-info">
**Note**: the update() function is required to apply the change.
</div>

In [35]:
data = me.process_data.complex_ab
me.reactions.a_to_b_FWD_complex_ab.complex_data = data
print('Forward reaction (before update): %s' %
      (me.reactions.a_to_b_FWD_complex_ab.reaction))
me.reactions.a_to_b_FWD_complex_ab.update()
print('Forward reaction (after update): %s' %
      (me.reactions.a_to_b_FWD_complex_ab.reaction))
print('')

me.reactions.a_to_b_REV_complex_ab.complex_data = data
print('Reverse reaction (before update): %s' %
      (me.reactions.a_to_b_REV_complex_ab.reaction))
me.reactions.a_to_b_REV_complex_ab.update()
print('Reverse reaction (after update): %s' %
      (me.reactions.a_to_b_REV_complex_ab.reaction))

Forward reaction (before update): a --> b
Forward reaction (after update): a + 4.27350427350427e-6*mu complex_ab --> b

Reverse reaction (before update): b --> a
Reverse reaction (after update): b + 4.27350427350427e-6*mu complex_ab --> a


The coefficient for complex_ab is determined by the expression $$\frac{\mu}{k_{eff}}$$ which in its entirety represents the dilution of an enzyme following a cell doubling. The coupling constraint can be summarized as followed
$$ 
\begin{align}
&v_{dilution,j} = \mu \sum_{i} \left( \frac{1}{k_{eff,i}} v_{usage,i} \right), & \forall j \in Enzyme
\end{align}
$$

Where 

 - $v_{usage,i}$ is the flux through the metabolic reaction
 - $k_{eff}$ is the turnover rate for the process and conveys the productivity of the enzyme complex. Physically, it can be thought of as the number of reactions the enzyme can catalyze per cell division. 


By default the $k_{eff}$ for a MetabolicReaction is set to 65 but this can be changed using the code below.

#### Different Keff for forward reaction

In [36]:
me.reactions.a_to_b_FWD_complex_ab.keff = .00001
me.reactions.a_to_b_FWD_complex_ab.update()

# The forward and reverse direction can have differing keffs
print('Forward reaction')
print('----------------')
print(me.reactions.a_to_b_FWD_complex_ab.reaction)
print('')
print('Reverse reaction')
print('----------------')
print(me.reactions.a_to_b_REV_complex_ab.reaction)

Forward reaction
----------------
a + 27.7777777777778*mu complex_ab --> b

Reverse reaction
----------------
b + 4.27350427350427e-6*mu complex_ab --> a


## Adding Reactions using utility functions
Add reactions using some of the utility functions provided in `cobrame.util.building.py`

### Transcription
Using the utility functions to create the TranscribedGene metabolite has the advantage of forcing the assignment of sequence, strand and RNA_type. 

In [37]:
building.create_transcribed_gene(me, 'b','tRNA', 'ATCG')
building.add_transcription_reaction(me, 'TU_b', {'b'}, sequence)
print(me.reactions.transcription_TU_b.reaction)
me.reactions.transcription_TU_b.update()

86 atp_c + 38 ctp_c + 12 gtp_c + 182 h2o_c + 50 utp_c --> RNA_b + 85 amp_c + 37 cmp_c + 11 gmp_c + 182 h_c + 186 ppi_c + 1.2817349999999998 tRNA_biomass + 49 ump_c


/home/sbrg-cjlloyd/cobrame/cobrame/core/reaction.py:813 UserWarning: RNA Polymerase () not found


### Translation
`add_translation_reaction` assumes that the RNA and protein have the same locus_id. It creates the appropriate `TranslationData` and `TranslationReaction` instance, links the two together and updates the `TranslationReaction`.

In [38]:
building.add_translation_reaction(me, 'b', dna_sequence=sequence, update=True)
print(me.reactions.translation_b.reaction)

/home/sbrg-cjlloyd/cobrame/cobrame/core/reaction.py:1094 UserWarning: RNA_degradosome not found


0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_b + 12 asp__L_c + 0.00448559670781893*(mu + 0.3915)/mu atp_c + 0.0076657950617284*(mu + 0.3915)/mu mRNA_biomass + met__L_c + 12 phe__L_c + 0.000874533914506385*mu + 0.00034238002752925 ribosome + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> 0.00448559670781893*(mu + 0.3915)/mu adp_c + 0.00598079561042524*(mu + 0.3915)/mu amp_c + 0.00598079561042524*(mu + 0.3915)/mu cmp_c + 0.00598079561042524*(mu + 0.3915)/mu gmp_c + 60.0 - 0.00448559670781893*(mu + 0.3915)/mu h2o_c + 0.00448559670781893*(mu + 0.3915)/mu h_c + 0.00448559670781893*(mu + 0.3915)/mu pi_c + protein_b + 7.500606 protein_biomass + 0.00598079561042524*(mu + 0.3915)/mu ump_c


### Complex Formation
Alternatively, ComplexData has a ```create_complex_formation()``` function to create the sythesis reaction following the naming conventions. It contains an ```update()``` function which incorporates changes in the ComplexData

In [39]:
data = cobrame.ComplexData('complex_ba', me)
data.stoichiometry = {'protein_a': 1, 'protein_b': 1}
data.create_complex_formation()
print(me.reactions.formation_complex_ba.reaction)

protein_a + protein_b --> complex_ba


### Metabolic Reaction

In [40]:
stoich_data = cobrame.StoichiometricData('b_to_c', me)
stoich_data._stoichiometry = {'b': -1, 'c': 1}
stoich_data.lower_bound = 0
stoich_data.upper_bound = 1000.
building.add_metabolic_reaction_to_model(me, stoich_data.id, 'forward', complex_id='complex_ab',
                                         update=True)
print('Reaction b_to_c')
print('---------------')
print(me.reactions.b_to_c_FWD_complex_ab.reaction)

Created <Metabolite c at 0x7f4bf6abf6d8> in <MetabolicReaction b_to_c_FWD_complex_ab at 0x7f4bf6abf7b8>
Reaction b_to_c
---------------
b + 4.27350427350427e-6*mu complex_ab --> c
